### Preprocess V4 reads

In [1]:
import glob
import pandas as pd
from Bio import SeqIO
from gzip import open as gopen
from collections import Counter

**Remove primers**

In [2]:
!mkdir -p cut_reads

for f in glob.glob('raw_reads/*_S*_L001_R*_001.fastq.gz'):
    outf = 'cut_reads/%s' % f.split('/')[-1]
    smpl = f.split('/')[-1].split('_')[0]
    records = SeqIO.parse(gopen(f,'rt'),'fastq')
    with gopen(outf,'wt') as hndl:
        for r in records:
            
            # Skip low quality reads
            counts = Counter(str(r[110:150].seq).upper())
            proportions = {i: counts[i]/sum(counts.values()) for i in counts.keys()}
            if any([proportions[i] > 0.5 for i in proportions.keys()]):
                continue
                
            hndl.write(r[20:150].format('fastq'))

**Merge reads**

In [3]:
!mkdir -p merged

df = pd.DataFrame(columns=['R1_count','R2_count',
                           'R1_mode_length','R1_count_at_mode_length',
                           'R2_mode_length','R2_count_at_mode_length',
                           'Mode_overlap','Count_at_mode_overlap'])

for f1 in glob.glob('cut_reads/*_S*_L001_R1_001.fastq.gz'):
    f2 = f1.replace('_R1_','_R2_')
    
    smpl = f1.split('/')[-1].split('_')[0]
    
    merged = gopen('merged/%s' % f1.split('/')[-1], 'wt')

    r1s = list(SeqIO.parse(gopen(f1,'rt'),'fastq'))
    r2s = SeqIO.to_dict(SeqIO.parse(gopen(f2,'rt'),'fastq'))
    
    count_r1 = len(r1s)
    count_r2 = len(r2s)
    
    overlapps = []
    r1_lengths= []
    r2_lengths= []
    
    for r1 in r1s:
        r1seq = str(r1.seq)
        r1_lengths.append(len(r1))
        try:
            r2seq = str(r2s[r1.id].seq.reverse_complement())
        except:
            continue
        r2_lengths.append(len(r2seq))
        over_len = 30
        r1_window = r1seq[-1*over_len:]
        r2_window = r2seq[:over_len]
        while over_len > 0 and r1_window != r2_window:
            over_len -= 1
            r1_window = r1seq[-1*over_len:]
            r2_window = r2seq[:over_len]
        overlapps.append(over_len)
        if over_len > 5:
            r2 = r2s[r1.id]
            m = r1 + r2.reverse_complement()[over_len:]
            m.id = r1.id
            m.description = r1.description
            merged.write(m.format('fastq'))
    merged.close()        
    if len(overlapps) == 0:
        print( smpl)
        continue
            
    ovlp_mode, ovlp_count_at_mode = sorted(Counter(overlapps).items(), key=lambda k: k[1], reverse=True)[0]
    r1len_mode, r1len_count_at_mode = sorted(Counter(r1_lengths).items(), key=lambda k: k[1], reverse=True)[0]
    r2len_mode, r2len_count_at_mode = sorted(Counter(r2_lengths).items(), key=lambda k: k[1], reverse=True)[0]
    df.loc[smpl] = [count_r1,count_r2,r1len_mode,r1len_count_at_mode,r2len_mode,r2len_count_at_mode,ovlp_mode,ovlp_count_at_mode]
df

Tank-13B


,R1_count,R2_count,R1_mode_length,R1_count_at_mode_length,R2_mode_length,R2_count_at_mode_length,Mode_overlap,Count_at_mode_overlap
AS2014,16994,16737,130,16994,130,16672,8,14023
AS2082,36080,36189,130,36080,130,35308,8,27719
Tank-2C,19112,19285,130,19112,130,18842,8,15469
AS2075,20743,20628,130,20743,130,20435,8,17617
AS2160,19792,19785,130,19792,130,19384,8,16245
...,...,...,...,...,...,...,...,...
AS2053,19038,19594,130,19038,130,18784,8,16371
AS2027,19872,19570,130,19872,130,19516,8,16415
AS2140,20159,20542,130,20159,130,19830,8,17236
AS2052,39063,38939,130,39063,130,38127,8,30735


In [4]:
df.to_csv('metadata/read_merging_stas.tsv', sep='\t')

**Format as qiime2 archive**

In [5]:
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path merged \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path merged/demux.qza

Imported merged as CasavaOneEightSingleLanePerSampleDirFmt to merged/demux.qza


**Run dada**  
No need to trim, primers were already removed

In [6]:
!mkdir -p dada
!qiime dada2 denoise-single \
  --i-demultiplexed-seqs merged/demux.qza \
  --p-trim-left 0 \
  --p-trunc-len 250 \
  --p-max-ee 5.0 \
  --o-representative-sequences dada/rep-seqs.qza \
  --o-table dada/table.qza \
  --o-denoising-stats dada/stats.qza

Saved FeatureTable[Frequency] to: dada/table.qza
Saved FeatureData[Sequence] to: dada/rep-seqs.qza
Saved SampleData[DADA2Stats] to: dada/stats.qza


In [7]:
!wget https://data.qiime2.org/2020.11/common/silva-138-99-515-806-nb-classifier.qza
!qiime feature-classifier classify-sklearn \
  --i-classifier silva-138-99-515-806-nb-classifier.qza \
  --i-reads dada/rep-seqs.qza \
  --o-classification dada/taxonomy.qza
!rm silva-138-99-515-806-nb-classifier.qz

--2021-04-02 15:40:13--  https://data.qiime2.org/2020.11/common/silva-138-99-515-806-nb-classifier.qza
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/common/silva-138-99-515-806-nb-classifier.qza [following]
--2021-04-02 15:40:14--  https://s3-us-west-2.amazonaws.com/qiime2-data/2020.11/common/silva-138-99-515-806-nb-classifier.qza
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.185.216
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.185.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153517385 (146M) [binary/octet-stream]
Saving to: ‘silva-138-99-515-806-nb-classifier.qza.1’

silva-138-99-515-80 100%[===================>] 146.41M  8.51MB/s    in 51s     

2021-04-02 15:41:07 (2.86 MB/s) - ‘silva-1

### make tree

In [8]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences dada/rep-seqs.qza \
  --o-alignment dada/aligned-rep-seqs.qza \
  --o-masked-alignment dada/masked-aligned-rep-seqs.qza \
  --o-tree dada/unrooted-tree.qza \
  --o-rooted-tree dada/rooted-tree.qza

Saved FeatureData[AlignedSequence] to: dada/aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: dada/masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: dada/unrooted-tree.qza
Saved Phylogeny[Rooted] to: dada/rooted-tree.qza
